In [6]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load model
https://huggingface.co/luqh/ClinicalT5-large

In [63]:
model_name = "luqh/ClinicalT5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name, model_max_length = 8192)
model = T5ForConditionalGeneration.from_pretrained(model_name, from_flax = True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
All Flax model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the Flax model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Some medical text

Not that this will fail if there are too many tokens. We'll have to do some chunking or RAG in order to
deal with the small context window.

In [64]:
medical_text = """
DESCRIPTION OF PROCEDURE:  ,After induction of general anesthesia, the patient was placed prone on the operating room table resting on chest rolls.  Her face was resting in a pink foam headrest.  Extreme care was taken positioning her because she weighs 92 kg.  There was a lot of extra padding for her limbs and her limbs were positioned comfortably.  The arms were not hyperextended.  Great care was taken with positioning of the head and making sure there was no pressure on her eyes especially since she already has visual disturbance.  A Foley catheter was in place.  She received IV Cipro 400 mg because she is allergic to most antibiotics.,Fluoroscopy was used to locate the lower end of the fractured catheter and the skin was marked.  It was also marked where we would try to insert the new catheter at the L4 or L3 interspinous space.,The patient was then prepped and draped in a sterile manner.,A 7-cm incision was made over the L1 lamina.  The incision was carried down through the fascia all the way down to the spinous processes.  A self-retaining McCullough retractor was placed.  The laminae were quite deep.  The microscope was brought in and using the Midas Rex drill with the AM-8 bit and removing some of the spinous process of L1-L2 with double-action rongeurs, the laminotomy was then done using the drill and great care was taken and using a 2-mm rongeur, the last layer of lamina was removed exposing the epidural fat and dura.  The opening in the bone was 1.5 x 1.5 cm.,Occasionally, bipolar cautery was used for bleeding of epidural veins, but this cautery was kept to a minimum.,Under high magnification, the dura was opened with an 11 blade and microscissors.  At first, there was a linear incision vertically to the left of midline, and I then needed to make a horizontal incision more towards the right.  The upper aspect of the cauda equina was visualized and perhaps the lower end of the conus.  Microdissection under high magnification did not expose the catheter.  The fluoroscope was brought in 2 more times including getting a lateral view and the fluoroscope appeared to show that the catheter should be in this location.,I persisted with intensive microdissection and finally we could see the catheter deep to the nerves and I was able to pull it out with the microforceps.,The wound was irrigated with bacitracin irrigation.,At this point, I then attempted lumbar puncture by making a small incision with an 11 blade in the L4 interspinous space and then later in the L3 interspinous space and attempted to puncture the dural sac with the Tuohy needle.  Dr. Y also tried.  Despite using the fluoroscope and our best attempts, we were not able to convincingly puncture the lumbar subarachnoid space and so the attempted placement of the new lumbar catheter had to be abandoned.  It will be done at a later date.,I felt it was unsafe to place a new catheter at this existing laminotomy site because it was very high up near the conus.  The potential for complications involving her spinal cord was greater and we have already had a complication of the catheter now and I just did not think it was safe to put in this location.,Under high magnification, the dura was closed with #6-0 PDS interrupted sutures.,After the dura was closed, a piece of Gelfoam was placed over the dura.  The paraspinous muscles were closed with 0 Vicryl interrupted sutures.  The subcutaneous fascia was also closed with 0 Vicryl interrupted suture.  The subcutaneous layer was closed with #2-0 Vicryl interrupted suture and the skin with #4-0 Vicryl Rapide.  The 4-0 Vicryl Rapide sutures were also used at the lumbar puncture sites to close the skin.,The patient was then turned carefully on to her bed after sterile dressings were applied and then taken to the recovery room.  The patient tolerated procedure well.  No complications.  Sponge and needle counts correct.  Blood loss minimal, none replaced.  This procedure took 5 hours.  This case was also extremely difficult due to patient's size and the difficulty of locating the catheter deep to the cauda equina.
"""

# Summarize

In [67]:
prompt = "summarize clinical note: " + medical_text.strip()
inputs = tokenizer(prompt, return_tensors = "pt", truncation = True)

with torch.no_grad():
    summary_ids = model.generate(
        **inputs,
        max_new_tokens = 128,
        min_length = 60,        
        num_beams = 5,
        length_penalty = 2.0,        # prefer longer response
        early_stopping = False,      # let all beams finish
        no_repeat_ngram_size = 3     # reduce verbatim copying
    )

In [68]:
summary = tokenizer.decode(summary_ids[0], skip_special_tokens = True)
print(summary)

- - - the patient was placed on a - she was placed prone on the operating room table resting on chest rolls with her head resting in a pink foam headrest. she was positioned comfortably. she had a she is very obese. she received she received the the the lamina was opened. the skin was marked.,     and and
